In [3]:
import dlt
from dlt.sources.helpers.rest_client.auth import APIKeyAuth
from dlt.sources.rest_api import RESTAPIConfig
from dlt.sources.rest_api import rest_api_source, rest_api_resources

In [4]:
auth = APIKeyAuth(name="X-API-Key", 
                   api_key=dlt.secrets["sources.bfrb.api_key"], 
                location="header") 
url=dlt.config["sources.bfrb.url"]

In [10]:
@dlt.source
def order_source():
    source: RESTAPIConfig = {
        "client": {
            "base_url": url,
            "auth": auth,
        }   ,
        "resources": [
            {
               "name": "store",
                "write_disposition": "replace",
                "endpoint": {
                    "path": "/global/store",
                },
            }
        ]
    }

    yield from rest_api_resources(source)

In [13]:
pipeline = dlt.pipeline(
    pipeline_name="jeudi_rb",
    destination=dlt.destinations.duckdb("Data/rb.db"),
    dataset_name="chick"
)

In [15]:
load_info = pipeline.run(order_source())

print(load_info)

2025-05-12 10:36:16,531|[WARNING]|22776|21164|dlt|client.py|detect_paginator:312|Fallback paginator used: SinglePagePaginator at 187ae1b9110. Please provide right paginator manually.


Pipeline jeudi_rb load step completed in 1.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset chick
The duckdb destination used duckdb:///c:\Dev\Lobellia\Data/rb.db location to store data
Load package 1747038976.3651254 is LOADED and contains no failed jobs


In [ ]:
def order_source():
    source: RESTAPIConfig = {
        "client": {
            "base_url": url,
            "auth": auth,
        }   ,
        "resources": [
            {
               "name": "store",
                "write_disposition": "replace",
                "endpoint": {
                    "path": "/global/store",
                },
            },
            {
                "name": "order_picture",
                "write_disposition": "apppend",
                "endpoint": {
                    "path": "/sale/reports/order-picture?store-id={store-id}",
                    "params": {
                        "timestamp-from": "20240101000000",   
                        "store-id": {
                            "type": "resolve",
                            "resource": "store",
                            "field": "id",
                        },
                    }   ,
                "data_selector": "order-pictures[*]"
                },
            }
 
    yield from rest_api_resources(source)

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name="jeudi_rb",
    destination=dlt.destinations.duckdb("Data/rb.db"),
    dataset_name="chick"
)